# Research Paper Analysis System

This notebook implements a system for analyzing research papers, specifically extracting hypotheses and identifying research gaps.

## Installation of Required Libraries

First, let's install the necessary dependencies:

In [ ]:
!pip install streamlit torch transformers peft langchain langchain-community faiss-cpu pypdf2 accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires reques

## Import Libraries

In [ ]:
import streamlit as st
import torch
import json
import tempfile
import os
import PyPDF2
import getpass
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline
)
from peft import PeftModel
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, HTML

## Set Up HuggingFace Token

Securely enter your HuggingFace token when prompted

In [ ]:
# Securely get HuggingFace token using getpass

import getpass
import os

hf_token = getpass.getpass("Enter your HuggingFace token: ")
os.environ["HF_TOKEN"] = hf_token


Enter your HuggingFace token: ··········


## Initialize Session Variables

In [ ]:
# Initialize variables to store state (replacing streamlit session state)
class SessionState:
    def __init__(self):
        self.model = None
        self.tokenizer = None
        self.embeddings = None
        self.vector_store = None
        self.pdf_text = ""
        self.hf_model_repo = None  # Model repository on HuggingFace
        self.hypotheses = None
        self.limitations = None

session_state = SessionState()

## Define Functions

In [ ]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

def load_model_and_tokenizer(model_repo):
    """Load the model and tokenizer from a Hugging Face repository"""

    # Set up the configuration for quantization
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    # Use HF_TOKEN from environment variables
    hf_token = os.environ.get("HF_TOKEN", None)

    if not hf_token:
        print("Warning: HuggingFace token not found. Some models may not load correctly.")

    # Base model fallback (public model)
    base_model_name = "tiiuae/falcon-7b-instruct"

    print("Loading model and tokenizer from HuggingFace repo...")

    try:
        # Try to load tokenizer from the fine-tuned model repo first
        tokenizer = AutoTokenizer.from_pretrained(model_repo, use_auth_token=hf_token)
        print(f"Successfully loaded tokenizer from {model_repo}")
    except Exception as e:
        print(f"Could not load tokenizer from {model_repo}. Using base model tokenizer instead. Error: {e}")
        tokenizer = AutoTokenizer.from_pretrained(base_model_name, use_auth_token=hf_token)

    # Make sure pad token is set
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    print(f"Loading model from {model_repo}... This may take a few minutes...")
    try:
        # Try to load the fine-tuned model directly from the repo
        model = AutoModelForCausalLM.from_pretrained(
            model_repo,
            device_map="auto",
            quantization_config=bnb_config,
            use_auth_token=hf_token
        )
        print(f"Successfully loaded model from {model_repo}")
    except Exception as e:
        print(f"Could not load complete model from {model_repo}. Error: {e}")
        print("Loading base model and then attempting to load adapters...")

        # If loading the whole model failed, try loading base model + adapters
        base_model = AutoModelForCausalLM.from_pretrained(
            base_model_name,
            device_map="auto",
            quantization_config=bnb_config,
            use_auth_token=hf_token
        )

        try:
            model = PeftModel.from_pretrained(base_model, model_repo, use_auth_token=hf_token)
            print(f"Successfully loaded adapter from {model_repo}")
        except Exception as adapter_error:
            print(f"Failed to load adapter from {model_repo}. Using base model. Error: {adapter_error}")
            model = base_model

    return model, tokenizer


In [ ]:
def extract_text_from_pdf(pdf_file_path):
    """Extract text from a PDF file"""
    text = ""
    with open(pdf_file_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()

    return text

In [ ]:
def create_vector_store(text):
    """Create a vector store from the text"""
    # Split text into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
    )
    chunks = text_splitter.split_text(text)

    # Load embeddings model
    if session_state.embeddings is None:
        print("Loading embeddings model...")
        session_state.embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2"
        )

    # Create vector store
    print("Creating vector store...")
    vector_store = FAISS.from_texts(chunks, session_state.embeddings)

    return vector_store, chunks

In [ ]:
def generate_hypothesis(abstract_text):
    """Agent 1: Generate null and alternate hypotheses from abstract"""

    instruction = "Extract the null and alternate hypotheses from the given abstract."
    prompt = f"### Instruction:\n{instruction}\n\n### Input:\n{abstract_text}\n\n### Response:\n"

    inputs = session_state.tokenizer(prompt, return_tensors="pt").to(session_state.model.device)

    outputs = session_state.model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=True,
        top_p=0.9,
        top_k=50,
        temperature=0.5
    )

    response = session_state.tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Try to extract just the response part
    if "### Response:" in response:
        response = response.split("### Response:")[1].strip()

    return response

In [ ]:
def identify_limitations(paper_text):
    """Agent 2: Identify limitations in the research paper"""

    instruction = """
You are an academic research analyst.

Task: From the following paper text, identify 3 to 6 specific research gaps and express them as clear, factual statements.

Requirements:
- Each point should be concise (1–2 lines).
- Avoid phrasing as questions. Use declarative statements.
- Each gap should highlight what is missing, underexplored, or limited in current research.
- Avoid repetition and generic phrases.
- Use bullet points starting with "-".
- Focus on clarity and insightfulness.
"""

    # If text is too long, use a summarized version or the abstract
    if len(paper_text) > 4000:
        # Try to find abstract section
        lower_text = paper_text.lower()
        if "abstract" in lower_text:
            start_idx = lower_text.find("abstract")
            end_idx = lower_text.find("introduction", start_idx)
            if end_idx == -1:  # If no "introduction" found
                end_idx = start_idx + 2000  # Use 2000 chars after abstract
            paper_text = paper_text[start_idx:end_idx]
        else:
            # Just use the first 4000 characters if no abstract found
            paper_text = paper_text[:4000]

    prompt = f"### Instruction:\n{instruction}\n\n### Input:\n{paper_text}\n\n### Response:\n"

    inputs = session_state.tokenizer(prompt, return_tensors="pt").to(session_state.model.device)

    outputs = session_state.model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=True,
        top_p=0.9,
        top_k=50,
        temperature=0.5,  # Slightly higher temperature for more creative analysis
        repetition_penalty=1.25,  # key fix for repetition
        num_return_sequences=1
        )

    response = session_state.tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Try to extract just the response part
    if "### Response:" in response:
        response = response.split("### Response:")[1].strip()

    return response

In [ ]:
def perform_rag_query(query, top_k=3):
    """Perform RAG query to find most relevant chunks for a query"""
    if session_state.vector_store is None:
        print("No vector store available. Please upload a PDF first.")
        return []

    # Get relevant documents
    results = session_state.vector_store.similarity_search(query, k=top_k)
    return [doc.page_content for doc in results]

## Specify Hugging Face Model Repository

In [ ]:
# Enter your HuggingFace model repository
model_repo = "tiiuae/falcon-7b-instruct"
session_state.hf_model_repo = model_repo

print(f"Using model from: {model_repo}")

Using model from: tiiuae/falcon-7b-instruct


## Load Model from HuggingFace Repository

In [ ]:
# Load model from Hugging Face repository
session_state.model, session_state.tokenizer = load_model_and_tokenizer(session_state.hf_model_repo)

Loading model and tokenizer from HuggingFace repo...


/usr/local/lib/python3.12/dist-packages/transformers/models/auto/tokenization_auto.py:1025: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

Successfully loaded model from tiiuae/falcon-7b-instruct


## Upload and Process PDF

In [ ]:
print("Please upload a research paper (PDF):")
uploaded = files.upload()

# Get the first file name from the uploaded files
if uploaded:
    file_name = list(uploaded.keys())[0]
    print(f"Processing {file_name}...")

    # Extract text from the PDF
    session_state.pdf_text = extract_text_from_pdf(file_name)

    # Create vector store
    session_state.vector_store, chunks = create_vector_store(session_state.pdf_text)

    print(f"Successfully processed PDF: {file_name}")
    print(f"Extracted {len(chunks)} text chunks")
    print("\nFirst 1000 characters of the extracted text:")
    print(session_state.pdf_text[:1000])

Please upload a research paper (PDF):


Saving d4va00136b.pdf to d4va00136b.pdf
Processing d4va00136b.pdf...
Loading embeddings model...


/tmp/ipython-input-3317436969.py:14: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  session_state.embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Creating vector store...
Successfully processed PDF: d4va00136b.pdf
Extracted 181 text chunks

First 1000 characters of the extracted text:
Innovative approaches to sustainable wastewater
treatment: a comprehensive exploration of
conventional and emerging technologies
Jaweria Shamshad *aand Rashid Ur Rehman *b
Access to clean water is under threat due to population growth, climate change, and pollution, emphasizing
the need for e ﬀective wastewater treatment. Wastewater pollutants pose risks to public health and
ecosystems, necessitating proper treatment methods. This paper outlines both conventional and emergingtechnologies for wastewater treatment. Established techniques, such as activated sludge processing,
chlorination, and constructed wetlands, are discussed alongside newer methods, such as advanced
oxidation, ultraviolet disinfection, membrane bioreactors, reverse osmosis, arti ﬁcial intelligence
optimization, and nano ﬁltration, which enhance contaminant removal but may incur hi

## Generate Hypotheses

In [ ]:
if session_state.model is not None and session_state.pdf_text:
    print("Generating hypotheses...")
    # For hypotheses, we'll use the first part of the paper which likely contains the abstract
    abstract_text = session_state.pdf_text[:3000]
    session_state.hypotheses = generate_hypothesis(abstract_text)

    print("\nHypotheses:")
    print(session_state.hypotheses)
else:
    print("Please make sure the model is loaded and a PDF has been processed before generating hypotheses.")

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Generating hypotheses...

Hypotheses:
The null hypothesis is that there are no significant differences in treatment efficiency between conventional and emerging technologies. The alternate hypothesis is that there are significant differences in treatment efficiency between these technologies.


## Identify Research Gaps

In [ ]:
if session_state.model is not None and session_state.pdf_text:
    print("Identifying research gaps...")
    # For limitations, use the RAG system to find relevant sections
    relevant_chunks = perform_rag_query("limitations methodology weaknesses future work future scope unexplored", top_k=5)
    combined_text = "\n".join(relevant_chunks)
    session_state.limitations = identify_limitations(combined_text)

    print("\nResearch Gaps:")
    print(session_state.limitations)
else:
    print("Please make sure the model is loaded and a PDF has been processed before identifying research gaps.")

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Identifying research gaps...

Research Gaps:
1. Research gap: The paper does not clearly state the purpose of the research. It should provide a clear statement of the research question or objective.

2. Research gap: The paper does not provide a clear statement of the research methods or the data sources used. It should clearly describe the methods and data sources used in the research.

3. Research gap: The paper does not provide a clear statement of the research limitations or challenges. It should clearly describe the limitations and challenges faced in the research.

4. Research gap: The paper does not provide a clear statement of the research implications. It should clearly describe the implications of the research.

5. Research gap: The paper does not provide a clear statement of the research limitations or challenges faced in the research. It should clearly describe the limitations and challenges faced in the research.

6. Research gap: The paper does not provide a clear stateme

In [ ]:
!pip install streamlit pyngrok langchain openai tiktoken unstructured faiss-cpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 19.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 105.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.8/207.8 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 11.5 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=d41230eb0ec69a171f5b0d86d6a515594eafb17fa09f54a2cb9e838e814b0a3c
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


In [ ]:
%%writefile app.py
import streamlit as st
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredPDFLoader

# Session state
if 'vector_store' not in st.session_state:
    st.session_state.vector_store = None

st.title("📄 RAG PDF Research Assistant")

# Upload PDF
uploaded_file = st.file_uploader("Upload a PDF", type="pdf")
if uploaded_file:
    with open("temp.pdf", "wb") as f:
        f.write(uploaded_file.read())

    loader = UnstructuredPDFLoader("temp.pdf")
    docs = loader.load()

    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    chunks = splitter.split_documents(docs)

    embeddings = OpenAIEmbeddings()
    vector_store = FAISS.from_documents(chunks, embeddings)
    st.session_state.vector_store = vector_store

    st.success("PDF uploaded and processed successfully!")

# Query
query = st.text_input("Enter your query:")
if st.button("Search") and query:
    if st.session_state.vector_store is None:
        st.warning("Please upload a PDF first!")
    else:
        results = st.session_state.vector_store.similarity_search(query, k=3)
        st.write("**Top Results:**")
        for i, doc in enumerate(results):
            st.write(f"{i+1}. {doc.page_content}")



Overwriting app.py


In [ ]:
!ngrok authtoken YOUR_AUTHTOKEN_HERE

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok
import os

# Run Streamlit in the background
get_ipython().system_raw("streamlit run app.py --server.port 8501 &")

# Create a public URL
public_url = ngrok.connect(port=8501)
public_url


ERROR:pyngrok.process.ngrok:t=2025-10-14T19:20:55+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: The authtoken you specified does not look like a proper ngrok authtoken.\nYour authtoken: YOUR_AUTHTOKEN_HERE\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n"


PyngrokNgrokError: The ngrok process errored on start: authentication failed: The authtoken you specified does not look like a proper ngrok authtoken.\nYour authtoken: YOUR_AUTHTOKEN_HERE\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n.

In [ ]:
!pip install nbconvert


In [ ]:
!jupyter nbconvert nlp_paper.ipynb --to notebook --ClearOutputPreprocessor.enabled=True --output cleaned_notebook.ipynb
